In [1]:
# Import PyTorch Modules that will be used in the lab

import torch 
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas
from torchvision import transforms
import torch.nn as nn
torch.manual_seed(0)

Import Non-PyTorch Modules 


In [2]:
# Import Non-PyTorch Modules that will be used in the lab

import time
from imageio import imread
from matplotlib.pyplot import imshow
import matplotlib.pylab as plt
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np

<hr>


In [7]:
# Url that contains CSV files and image dataset folder

test_csv_file = 'data/test_labels.csv'
test_data_dir = 'data/test_data_pytorch/'

In [8]:
# Create Dataset Class

class Dataset(Dataset):
    
    # Constructor
    def __init__(self, csv_file, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.data_name = pd.read_csv(csv_file)
        self.len = self.data_name.shape[0] 
    
    # Get Length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
        img_name = self.data_dir + self.data_name.iloc[idx, 2]
        image = Image.open(img_name)
        y = self.data_name.iloc[idx, 3]
        if self.transform:
            image = self.transform(image)
        return image, y

In [9]:
# Construct the composed object for transforming the image 
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
trans_step = [transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize(mean, std)]


composed = transforms.Compose(trans_step)

In [10]:
# Create a test_dataset

test_dataset = Dataset(transform=composed
                       , csv_file=test_csv_file
                       , data_dir=test_data_dir)

<hr>


In [11]:
# Load pre-trained model
model = torch.load("resnet18_pytorch.pt")
model_des = torch.load("densenet121_pytorch.pt")

In [12]:
# Print model structure

print("ResNet18:\n", model)
print("Densenet121:\n", model_des)

ResNet18:
 ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inpla

In [13]:
# Set Data Loader object

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=10)

<hr>


In [18]:
# Predict the data using ResNet18 model and print out accuracy

correct=0
accuracy=0
length = len(test_dataset)
for X_test,y_test in test_loader:
    model.eval()
    z = model(X_test)
    _,pred = torch.max(z.data,1)
    correct += (pred==y_test).sum().item()
    accuracy = correct / length
print(f'Accuracy in Resnet18: {accuracy}')

Accuracy in Resnet18: 0.11428571428571428


In [19]:
# Predict the data using densene model and print out accuracy

correct=0
accuracy=0
length = len(test_dataset)

for X_test,y_test in test_loader:
    model_des.eval()
    z = model_des(X_test)
    _,pred = torch.max(z.data,1)
    correct += (pred == y_test).sum().item()
    accuracy = correct / length
print(f'Accuracy in densenet121 {accuracy}')

    

Accuracy in densenet121 0.18571428571428572
